## 1. Data Pre-Processing¶

### 1-1. Read Data

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import random
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)
%%time

In [ ]:
X=[]
y=[]

path='/content/gdrive/My Drive/Colab Notebooks/101_ObjectCategories'
category_list=os.listdir(path)
for category in tqdm(category_list):
    imges=os.listdir(path+'/'+category)
    for img in imges:
        data=cv2.imread(path+'/'+category+'/'+img)
        data=cv2.resize(data,(224,224))
        label=category
        X.append(data)
        y.append(label)

In [ ]:
X=np.array(X)
y=np.array(y)
encoder=LabelEncoder()
labels=encoder.fit_transform(y)

In [ ]:
from keras.utils import to_categorical

display(labels.shape)

In [ ]:
labels = to_categorical(labels, num_classes=None)
display(labels.shape)

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(X[0])
plt.title(y[0])
plt.show()

### 1-2. Processing

In [ ]:
X= X/255
from sklearn.model_selection import train_test_split

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X,labels,test_size=0.2,random_state=7)
train_data, validation_data, train_label, validation_label = train_test_split(X,labels,test_size=0.2,random_state=7)
del X

In [ ]:
#Augmentation
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

In [ ]:
datagen.fit(train_data)

## 2. Modeling

### 2-1. ResNet

In [ ]:
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
from keras import optimizers
from keras.optimizers import adam
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

In [ ]:
resnet_not_trained = ResNet50(include_top=True,input_shape=(224,224,3))
model = models.Sequential()
model.add(resnet_not_trained)
model.add(layers.Dense(102,activation = 'softmax'))
model.summary()
batch_size = 32

In [ ]:
history = model.fit_generator(datagen.flow(train_data, train_label,
                                           batch_size=batch_size),
                              steps_per_epoch= (train_data.shape[0] // batch_size),
                              epochs = 50,
                              validation_data=(validation_data,validation_label))

## 3. Transfer Learning

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

resnet = ResNet50(weights = 'imagenet', include_top = False, input_shape = (224,224,3))
model = models.Sequential()
model.add(resnet)
model.add(layers.Flatten())
model.add(layers.Dense(102,activation = 'softmax'))
model.summary()

In [ ]:
# resnet를 동결
print('resnet 동결하기 전 훈련되는 레이어의 수 :',len(model.trainable_weights))
resnet.trainable = False
print('resnet을 동결한 후 훈련되는 레이어의 수:',len(model.trainable_weights))

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = adam(),
             metrics = ['acc'])
batch_size = 32

In [ ]:
history = model.fit_generator(datagen.flow(train_data, train_label,
                                           batch_size=batch_size),
                              steps_per_epoch= (train_data.shape[0] // batch_size),
                              epochs = 50,
                              validation_data=(validation_data,validation_label))